In [19]:
import pandas as pd

data = pd.read_pickle('/home/p1dg/generation_bot/data/datasets/prepaired_data.df')
data = data[~(data['companions'].apply(len) == 0)].reset_index(drop=True)

In [20]:
import re
#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('', '', text)

def convert2several_words(text):
  new_text = ''
  for word in text.split():
    if not word.isupper():
      word = "".join((" " + w if (w.isupper() and i) else w) for i, w in enumerate(word))
      new_text+= " " + word
    else:
      new_text+=' '+word
  return ' '.join(new_text.split())

#Collect all functions
def clean_text(text):
    text = remove_between_square_brackets(text)
    text = convert2several_words(text)
    return ' '.join(text.split())

In [21]:
sentence = data.sample()["context"].values[0]
print(sentence)
print()
print(clean_text(sentence))

Of course I fear for the safety of my son!. And that is why we have to stop them.. The knights of the Vale will stay in the Vale where they belong, to protect their Lord.. Uncle. Father.. The rumors of your demise were unfounded.

Of course I fear for the safety of my son!. And that is why we have to stop them.. The knights of the Vale will stay in the Vale where they belong, to protect their Lord.. Uncle. Father.. The rumors of your demise were unfounded.


In [22]:
train_df = data[["speaker"]].reset_index(drop=True)
train_df["query"] = data["query"].apply(clean_text)

In [23]:
import random
use_col = ['speaker', 'query', 'context', "companions", "close_reply"]

train_df = data.iloc[:-5][use_col]

for index, row in data.iloc[:-5:,:].iterrows():
    train_df.loc[index,"query"] = clean_text(row["query"])
    train_df.loc[index, "context"] = clean_text(row["context"])
    id = random.randint(0, len(row["companions"]) - 1)
    train_df.loc[index, "companions"] = row["companions"][id]
    train_df.loc[index, "close_reply"] = clean_text(row["close_reply"][id])

In [24]:
train_df

,speaker,query,context,companions,close_reply
0,royce,What is it?,Ned. Ten is too young to see such things.. He ...,jon snow,You did well.
1,jon snow,And your mother.,He won't be a boy forever. And winter is comin...,eddard stark,Lord of the Seven Kingdoms and protector of th...
2,septa mordane,"Fine work, as always. Well done.","Lad, go run back and get the rest.. In the nam...",bran stark,Jon said he was a deserter.
3,sansa stark,Thank you.,"In the name of Robert of the House Baratheon, ...",bran stark,Jon said he was a deserter.
4,septa mordane,I love the detail that you've managed to get i...,Don't look away.. King of the Andals and the F...,eddard stark,The man who passes the sentence should swing t...
...,...,...,...,...,...
16154,tyrion lannister,Perhaps we should take shelter.,Long may he reign.. Long may he reign.. Long m...,tyrion lannister,"Ser Bronn of the Blackwater, Lord of Highgarde..."
16155,daenerys targaryen,The city is on the rise?,Long may he reign.. That will improve.. I'm su...,davos,We have. These projects will begin as soon as ...
16156,tyrion lannister,it proves that no one needs a Master.,That will improve.. I'm sure it will.. Ser Bro...,davos,Any more.
16157,daenerys targaryen,Good. Shall we begin?,I'm sure it will.. Ser Bronn of the Blackwater...,tyrion lannister,"Grandmaester, ahem, it is my theory, based on ..."


In [25]:
def make_prompt(df):
  id = random.randint(0, len(df[use_col]["companions"]))
  query = df['query']
  role = df['companions']
  context = df['context']
  reply = df["close_reply"]
  prompt = f"[INST]"
  prompt += f'Use the given context to guide your an about the query like indicated in your role'
  prompt += f"query: {query}\n\n"
  prompt += f"context: {context}\n\n"
  prompt += f"your role: {role}\n\n"
  prompt += f'answer:{reply}[/INST]'
  return prompt

for index, row in train_df.iterrows():
   train_df.loc[index, "prompt"] = make_prompt(row)

In [26]:

train_df.loc[10]["prompt"]
     

"[INST]Use the given context to guide your an about the query like indicated in your rolequery: Tell Bran he's coming, too.\n\ncontext: Is it true he saw the White Walkers?. The White Walkers have been gone for thousands of years.. So he was lying?. A madman sees what he sees.. What is it?\n\nyour role: eddard stark\n\nanswer:Tough old beast.[/INST]"

In [30]:
import datasets


dataset =  datasets.Dataset.from_pandas(train_df[["prompt"]] )

In [31]:

dataset[:3]

{'prompt': ["[INST]Use the given context to guide your an about the query like indicated in your rolequery: What is it?\n\ncontext: Ned. Ten is too young to see such things.. He won't be a boy forever. And winter is coming.. Lad, go run back and get the rest.. In the name of Robert of the House Baratheon, first of his name …. Don't look away.\n\nyour role: jon snow\n\nanswer:You did well.[/INST]",
  "[INST]Use the given context to guide your an about the query like indicated in your rolequery: And your mother.\n\ncontext: He won't be a boy forever. And winter is coming.. Lad, go run back and get the rest.. In the name of Robert of the House Baratheon, first of his name …. Don't look away.. King of the Andals and the First Men …\n\nyour role: eddard stark\n\nanswer:Lord of the Seven Kingdoms and protector of the realm, I, Eddard of the House Stark, Lord of Winterfell and Warden of the North, sentence you to die.[/INST]",
  "[INST]Use the given context to guide your an about the query li

In [32]:
dataset.save_to_disk('/home/p1dg/generation_bot/data/datasets/prepaire_dataset')

Saving the dataset (1/1 shards): 100%|██████████| 16159/16159 [00:00<00:00, 1234508.63 examples/s]


In [33]:
loaded = datasets.load_from_disk('/home/p1dg/generation_bot/data/datasets/prepaire_dataset')
loaded[:5]

{'prompt': ["[INST]Use the given context to guide your an about the query like indicated in your rolequery: What is it?\n\ncontext: Ned. Ten is too young to see such things.. He won't be a boy forever. And winter is coming.. Lad, go run back and get the rest.. In the name of Robert of the House Baratheon, first of his name …. Don't look away.\n\nyour role: jon snow\n\nanswer:You did well.[/INST]",
  "[INST]Use the given context to guide your an about the query like indicated in your rolequery: And your mother.\n\ncontext: He won't be a boy forever. And winter is coming.. Lad, go run back and get the rest.. In the name of Robert of the House Baratheon, first of his name …. Don't look away.. King of the Andals and the First Men …\n\nyour role: eddard stark\n\nanswer:Lord of the Seven Kingdoms and protector of the realm, I, Eddard of the House Stark, Lord of Winterfell and Warden of the North, sentence you to die.[/INST]",
  "[INST]Use the given context to guide your an about the query li